In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import warnings
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score, davies_bouldin_score
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/match-making/test (1).csv


In [2]:
df = pd.read_csv("../input/match-making/test (1).csv")
df.head()

id            name  age  angkatan  gender  \
0  rec_cq2iokrtlf0u2vlfs7e0      finally to   18        24    male   
1  rec_cq2iokrtlf0u2vlfs7eg     abdicate er   17        24  female   
2  rec_cq2iokrtlf0u2vlfs7f0   gutter hourly   17        24    male   
3  rec_cq2iokrtlf0u2vlfs7fg  brightly solid   18        24  female   
4  rec_cq2iokrtlf0u2vlfs7g0       mint king   18        23  female   

   looking_for_partner                         hobbies  \
0                 True        ["sports","golf","swim"]   
1                False            ["travel","writing"]   
2                 True  ["travel","cooking","reading"]   
3                 True       ["swim","code","writing"]   
4                 True            ["sports","reading"]   

                           interests               xata.createdAt  \
0                ["politics","game"]  2024-07-03T10:47:47.165022Z   
1               ["music","language"]  2024-07-03T10:47:47.165022Z   
2  ["art","game","history","movies"]  2024-07-03T10:47:47.165022Z   
3             ["politics","history"]  2024-07-03T10:47:47.165022Z   
4        ["art","language","movies"]  2024-07-03T10:47:47.165022Z   

                xata.updatedAt  xata.version  
0  2024-07-03T11:07:48.812423Z            17  
1  2024-07-03T11:09:14.227559Z            10  
2  2024-07-03T11:11:30.062533Z            12  
3  2024-07-03T11:07:58.014051Z            18  
4  2024-07-03T11:11:24.499794Z            14

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   50 non-null     object
 1   name                 50 non-null     object
 2   age                  50 non-null     int64 
 3   angkatan             50 non-null     int64 
 4   gender               50 non-null     object
 5   looking_for_partner  50 non-null     bool  
 6   hobbies              50 non-null     object
 7   interests            50 non-null     object
 8   xata.createdAt       50 non-null     object
 9   xata.updatedAt       50 non-null     object
 10  xata.version         50 non-null     int64 
dtypes: bool(1), int64(3), object(7)
memory usage: 4.1+ KB


In [4]:
mapping = {
    'male': 1,
    'female': 0
}
df['gender'] = df['gender'].map(mapping)


In [5]:
df['hobbies'].value_counts()

hobbies
["travel"]                         5
["sing"]                           4
["dance"]                          3
["writing"]                        2
["gym"]                            2
["code","sing"]                    2
["cooking"]                        2
["sports","reading"]               2
["reading","writing","sports"]     1
["code"]                           1
["sports","swim"]                  1
["reading","writing"]              1
["golf","swim","travel"]           1
["reading","cooking"]              1
["gym cooking"]                    1
["dance","writing"]                1
["gym","sports"]                   1
["soccer","sports"]                1
["code","swim"]                    1
["travel","dance"]                 1
["sports","golf","swim"]           1
["dance","sing"]                   1
["travel","golf","writing"]        1
["travel","writing"]               1
["dance","cooking","travel"]       1
["swim"]                           1
["golf"]                      

In [6]:
import ast

unique_hobbies = set()
for hobbies in df['hobbies']:
    unique_hobbies.update(ast.literal_eval(hobbies))

unique_hobbies = sorted(list(unique_hobbies))

# Create binary vectors for hobbies
def create_hobbies_vector(hobbies, unique_hobbies):
    hobbies_set = set(ast.literal_eval(hobbies))
    return [1 if hobby in hobbies_set else 0 for hobby in unique_hobbies]

hobbies_vectors = df['hobbies'].apply(lambda x: create_hobbies_vector(x, unique_hobbies))

# Create a DataFrame for the binary vectors
hobbies_df = pd.DataFrame(hobbies_vectors.tolist(), columns=unique_hobbies)

# Combine with the original DataFrame (excluding the original hobbies column)
df = df.drop('hobbies', axis=1).join(hobbies_df)
df

id                      name  age  angkatan  gender  \
0   rec_cq2iokrtlf0u2vlfs7e0                finally to   18        24       1   
1   rec_cq2iokrtlf0u2vlfs7eg               abdicate er   17        24       0   
2   rec_cq2iokrtlf0u2vlfs7f0             gutter hourly   17        24       1   
3   rec_cq2iokrtlf0u2vlfs7fg            brightly solid   18        24       0   
4   rec_cq2iokrtlf0u2vlfs7g0                 mint king   18        23       0   
5   rec_cq2iokrtlf0u2vlfs7gg           stereotype till   18        24       1   
6   rec_cq2iokrtlf0u2vlfs7h0                    oh not   19        23       0   
7   rec_cq2iokrtlf0u2vlfs7hg             grotesque yum   18        24       1   
8   rec_cq2iokrtlf0u2vlfs7i0           reassuringly ha   18        24       0   
9   rec_cq2iokrtlf0u2vlfs7ig                   ack bob   17        24       1   
10  rec_cq2iokrtlf0u2vlfs7j0       indeed courageously   19        22       0   
11  rec_cq2iokrtlf0u2vlfs7jg                  eek sick   18        24       1   
12  rec_cq2iokrtlf0u2vlfs7k0            whirlwind yuck   18        23       0   
13  rec_cq2iokrtlf0u2vlfs7kg           boastfully from   20        22       1   
14  rec_cq2iokrtlf0u2vlfs7l0         violently flowery   18        24       1   
15  rec_cq2iokrtlf0u2vlfs7lg       anxiously biosphere   16        24       0   
16  rec_cq2iokrtlf0u2vlfs7m0              sleepily meh   19        23       1   
17  rec_cq2iokrtlf0u2vlfs7mg                 wise able   18        24       0   
18  rec_cq2iokrtlf0u2vlfs7n0         incentive crinkle   19        24       0   
19  rec_cq2iokrtlf0u2vlfs7ng            yellow receipt   18        24       1   
20  rec_cq2iokrtlf0u2vlfs7o0  replica enthusiastically   21        21       0   
21  rec_cq2iokrtlf0u2vlfs7og              blah species   17        24       1   
22  rec_cq2iokrtlf0u2vlfs7p0            downturn miter   17        24       0   
23  rec_cq2iokrtlf0u2vlfs7pg           upbeat urgently   17        24       1   
24  rec_cq2iokrtlf0u2vlfs7q0             red sorrowful   18        24       0   
25  rec_cq2iokrtlf0u2vlfs7qg          phooey meanwhile   18        24       1   
26  rec_cq2iokrtlf0u2vlfs7r0      resemblance aspirate   18        24       0   
27  rec_cq2iokrtlf0u2vlfs7rg               bob readily   16        24       1   
28  rec_cq2iokrtlf0u2vlfs7s0              club culture   18        24       0   
29  rec_cq2iokrtlf0u2vlfs7sg                  mean boo   19        23       1   
30  rec_cq2iokrtlf0u2vlfs7t0    necessity decentralize   18        24       0   
31  rec_cq2iokrtlf0u2vlfs7tg                nor though   18        23       1   
32  rec_cq2iokrtlf0u2vlfs7u0             besides union   18        24       0   
33  rec_cq2iokrtlf0u2vlfs7ug          genre arrogantly   19        24       1   
34  rec_cq2iokrtlf0u2vlfs7v0             baseball toad   18        24       1   
35  rec_cq2iokrtlf0u2vlfs7vg                  jell yet   17        24       0   
36  rec_cq2iokrtlf0u2vlfs800          hideous sombrero   17        24       1   
37  rec_cq2iokrtlf0u2vlfs80g       optimistically drat   17        23       1   
38  rec_cq2iokrtlf0u2vlfs810                  milk dob   21        22       0   
39  rec_cq2iokrtlf0u2vlfs81g            lard judicious   20        22       1   
40  rec_cq2iokrtlf0u2vlfs820                eek rarely   18        24       0   
41  rec_cq2iokrtlf0u2vlfs82g            yuck monstrous   17        24       1   
42  rec_cq2iokrtlf0u2vlfs830         dental colorfully   18        23       1   
43  rec_cq2iokrtlf0u2vlfs83g       cappuccino provided   18        24       0   
44  rec_cq2iokrtlf0u2vlfs840           ladder accustom   17        24       1   
45  rec_cq2iokrtlf0u2vlfs84g    evergreen victoriously   18        24       1   
46  rec_cq2iokrtlf0u2vlfs850         usually intensely   19        23       0   
47  rec_cq2iokrtlf0u2vlfs85g           almost reawaken   18        24       0   
48  rec_cq2iokrtlf0u2vlfs860            jealously hide   

In [7]:
df['interests'].value_counts()

interests
["music"]                             5
["game"]                              4
["politics"]                          3
["game","language"]                   3
["history","movies"]                  3
["game","history"]                    3
["movies"]                            3
["art"]                               3
["music","art","language"]            2
["politics","history"]                2
["language"]                          2
["politics","game"]                   1
["politics","music","art"]            1
["language","movies"]                 1
["movies","art"]                      1
["music","history"]                   1
["politics","game","language"]        1
["history"]                           1
["art","history"]                     1
["music","game","movies"]             1
["music","language"]                  1
["music","art"]                       1
["politics","history","language"]     1
["art","movies"]                      1
["politics","art","game","movi

In [8]:
unique_interests = set()
for interests in df['interests']:
    unique_interests.update(ast.literal_eval(interests))

unique_interests = sorted(list(unique_interests))

def create_interests_vector(interests, unique_interests):
    interests_set = set(ast.literal_eval(interests))
    return [1 if interest in interests_set else 0 for interest in unique_interests]

interests_vectors = df['interests'].apply(lambda x: create_interests_vector(x, unique_interests))

interests_df = pd.DataFrame(interests_vectors.tolist(), columns=unique_interests)
df = df.drop('interests', axis=1).join(interests_df)
df

id                      name  age  angkatan  gender  \
0   rec_cq2iokrtlf0u2vlfs7e0                finally to   18        24       1   
1   rec_cq2iokrtlf0u2vlfs7eg               abdicate er   17        24       0   
2   rec_cq2iokrtlf0u2vlfs7f0             gutter hourly   17        24       1   
3   rec_cq2iokrtlf0u2vlfs7fg            brightly solid   18        24       0   
4   rec_cq2iokrtlf0u2vlfs7g0                 mint king   18        23       0   
5   rec_cq2iokrtlf0u2vlfs7gg           stereotype till   18        24       1   
6   rec_cq2iokrtlf0u2vlfs7h0                    oh not   19        23       0   
7   rec_cq2iokrtlf0u2vlfs7hg             grotesque yum   18        24       1   
8   rec_cq2iokrtlf0u2vlfs7i0           reassuringly ha   18        24       0   
9   rec_cq2iokrtlf0u2vlfs7ig                   ack bob   17        24       1   
10  rec_cq2iokrtlf0u2vlfs7j0       indeed courageously   19        22       0   
11  rec_cq2iokrtlf0u2vlfs7jg                  eek sick   18        24       1   
12  rec_cq2iokrtlf0u2vlfs7k0            whirlwind yuck   18        23       0   
13  rec_cq2iokrtlf0u2vlfs7kg           boastfully from   20        22       1   
14  rec_cq2iokrtlf0u2vlfs7l0         violently flowery   18        24       1   
15  rec_cq2iokrtlf0u2vlfs7lg       anxiously biosphere   16        24       0   
16  rec_cq2iokrtlf0u2vlfs7m0              sleepily meh   19        23       1   
17  rec_cq2iokrtlf0u2vlfs7mg                 wise able   18        24       0   
18  rec_cq2iokrtlf0u2vlfs7n0         incentive crinkle   19        24       0   
19  rec_cq2iokrtlf0u2vlfs7ng            yellow receipt   18        24       1   
20  rec_cq2iokrtlf0u2vlfs7o0  replica enthusiastically   21        21       0   
21  rec_cq2iokrtlf0u2vlfs7og              blah species   17        24       1   
22  rec_cq2iokrtlf0u2vlfs7p0            downturn miter   17        24       0   
23  rec_cq2iokrtlf0u2vlfs7pg           upbeat urgently   17        24       1   
24  rec_cq2iokrtlf0u2vlfs7q0             red sorrowful   18        24       0   
25  rec_cq2iokrtlf0u2vlfs7qg          phooey meanwhile   18        24       1   
26  rec_cq2iokrtlf0u2vlfs7r0      resemblance aspirate   18        24       0   
27  rec_cq2iokrtlf0u2vlfs7rg               bob readily   16        24       1   
28  rec_cq2iokrtlf0u2vlfs7s0              club culture   18        24       0   
29  rec_cq2iokrtlf0u2vlfs7sg                  mean boo   19        23       1   
30  rec_cq2iokrtlf0u2vlfs7t0    necessity decentralize   18        24       0   
31  rec_cq2iokrtlf0u2vlfs7tg                nor though   18        23       1   
32  rec_cq2iokrtlf0u2vlfs7u0             besides union   18        24       0   
33  rec_cq2iokrtlf0u2vlfs7ug          genre arrogantly   19        24       1   
34  rec_cq2iokrtlf0u2vlfs7v0             baseball toad   18        24       1   
35  rec_cq2iokrtlf0u2vlfs7vg                  jell yet   17        24       0   
36  rec_cq2iokrtlf0u2vlfs800          hideous sombrero   17        24       1   
37  rec_cq2iokrtlf0u2vlfs80g       optimistically drat   17        23       1   
38  rec_cq2iokrtlf0u2vlfs810                  milk dob   21        22       0   
39  rec_cq2iokrtlf0u2vlfs81g            lard judicious   20        22       1   
40  rec_cq2iokrtlf0u2vlfs820                eek rarely   18        24       0   
41  rec_cq2iokrtlf0u2vlfs82g            yuck monstrous   17        24       1   
42  rec_cq2iokrtlf0u2vlfs830         dental colorfully   18        23       1   
43  rec_cq2iokrtlf0u2vlfs83g       cappuccino provided   18        24       0   
44  rec_cq2iokrtlf0u2vlfs840           ladder accustom   17        24       1   
45  rec_cq2iokrtlf0u2vlfs84g    evergreen victoriously   18        24       1   
46  rec_cq2iokrtlf0u2vlfs850         usually intensely   19        23       0   
47  rec_cq2iokrtlf0u2vlfs85g           almost reawaken   18        24       0   
48  rec_cq2iokrtlf0u2vlfs860            jealously hide   

In [9]:
data = df.copy()
data.drop('xata.createdAt', axis=1, inplace=True)
data.drop('xata.updatedAt', axis=1, inplace=True)
data.drop('xata.version', axis=1, inplace=True)
data.drop('name', axis=1, inplace=True)
data.head()

id  age  angkatan  gender  looking_for_partner  code  \
0  rec_cq2iokrtlf0u2vlfs7e0   18        24       1                 True     0   
1  rec_cq2iokrtlf0u2vlfs7eg   17        24       0                False     0   
2  rec_cq2iokrtlf0u2vlfs7f0   17        24       1                 True     0   
3  rec_cq2iokrtlf0u2vlfs7fg   18        24       0                 True     1   
4  rec_cq2iokrtlf0u2vlfs7g0   18        23       0                 True     0   

   cooking  dance  golf  gym  ...  swim  travel  writing  art  game  history  \
0        0      0     1    0  ...     1       0        0    0     1        0   
1        0      0     0    0  ...     0       1        1    0     0        0   
2        1      0     0    0  ...     0       1        0    1     1        1   
3        0      0     0    0  ...     1       0        1    0     0        1   
4        0      0     0    0  ...     0       0        0    1     0        0   

   language  movies  music  politics  
0         0       0      0         1  
1         1       0      1         0  
2         0       1      0         0  
3         0       0      0         1  
4         1       1      0         0  

[5 rows x 25 columns]

In [10]:
ids = data['id']
rep_id = {id_: i+1 for i, id_ in enumerate(ids)}
data['id'] = df['id'].replace(rep_id)
data.head()

id  age  angkatan  gender  looking_for_partner  code  cooking  dance  golf  \
0   1   18        24       1                 True     0        0      0     1   
1   2   17        24       0                False     0        0      0     0   
2   3   17        24       1                 True     0        1      0     0   
3   4   18        24       0                 True     1        0      0     0   
4   5   18        23       0                 True     0        0      0     0   

   gym  ...  swim  travel  writing  art  game  history  language  movies  \
0    0  ...     1       0        0    0     1        0         0       0   
1    0  ...     0       1        1    0     0        0         1       0   
2    0  ...     0       1        0    1     1        1         0       1   
3    0  ...     1       0        1    0     0        1         0       0   
4    0  ...     0       0        0    1     0        0         1       1   

   music  politics  
0      0         1  
1      1         0  
2      0         0  
3      0         1  
4      0         0  

[5 rows x 25 columns]

In [11]:
extra = data['looking_for_partner']
data.drop('looking_for_partner', axis=1, inplace=True)
data.head()

id  age  angkatan  gender  code  cooking  dance  golf  gym  gym cooking  \
0   1   18        24       1     0        0      0     1    0            0   
1   2   17        24       0     0        0      0     0    0            0   
2   3   17        24       1     0        1      0     0    0            0   
3   4   18        24       0     1        0      0     0    0            0   
4   5   18        23       0     0        0      0     0    0            0   

   ...  swim  travel  writing  art  game  history  language  movies  music  \
0  ...     1       0        0    0     1        0         0       0      0   
1  ...     0       1        1    0     0        0         1       0      1   
2  ...     0       1        0    1     1        1         0       1      0   
3  ...     1       0        1    0     0        1         0       0      0   
4  ...     0       0        0    1     0        0         1       1      0   

   politics  
0         1  
1         0  
2         0  
3         1  
4         0  

[5 rows x 24 columns]

In [12]:
features = data.copy()
features.drop('id',axis=1,inplace=True)
# Determine the optimal number of clusters using the elbow method
inertia = []
K = range(1, 17)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features)
    inertia.append(kmeans.inertia_)
    
plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

In [13]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

optimal_clusters = 12

features = data.drop('id', axis=1)
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(features)

# Optional: Visualize the clustering (using PCA for dimensionality reduction)
#pca = PCA(n_components=2)
#principal_components = pca.fit_transform(features)
#pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
#pca_df['cluster'] = data['cluster']
data.head()


id  age  angkatan  gender  code  cooking  dance  golf  gym  gym cooking  \
0   1   18        24       1     0        0      0     1    0            0   
1   2   17        24       0     0        0      0     0    0            0   
2   3   17        24       1     0        1      0     0    0            0   
3   4   18        24       0     1        0      0     0    0            0   
4   5   18        23       0     0        0      0     0    0            0   

   ...  travel  writing  art  game  history  language  movies  music  \
0  ...       0        0    0     1        0         0       0      0   
1  ...       1        1    0     0        0         1       0      1   
2  ...       1        0    1     1        1         0       1      0   
3  ...       0        1    0     0        1         0       0      0   
4  ...       0        0    1     0        0         1       1      0   

   politics  cluster  
0         1        1  
1         0        5  
2         0        3  
3         1        6  
4         0       11  

[5 rows x 25 columns]

In [14]:
data['cluster'].value_counts()

cluster
1     7
4     6
8     6
3     5
9     5
6     4
10    4
5     3
2     3
0     3
11    2
7     2
Name: count, dtype: int64

In [15]:
print(data[data['cluster'] == 1])

    id  age  angkatan  gender  code  cooking  dance  golf  gym  gym cooking  \
0    1   18        24       1     0        0      0     1    0            0   
11  12   18        24       1     0        1      0     0    0            0   
14  15   18        24       1     1        0      0     1    0            0   
19  20   18        24       1     0        0      0     1    0            0   
25  26   18        24       1     0        0      0     1    0            0   
33  34   19        24       1     0        0      0     0    0            1   
36  37   17        24       1     0        0      0     1    0            0   

    ...  travel  writing  art  game  history  language  movies  music  \
0   ...       0        0    0     1        0         0       0      0   
11  ...       0        0    0     1        0         0       0      0   
14  ...       0        0    0     0        0         0       0      1   
19  ...       0        0    0     0        0         0       1      0   
25

In [16]:
print(data[data['cluster'] == 11])

    id  age  angkatan  gender  code  cooking  dance  golf  gym  gym cooking  \
4    5   18        23       0     0        0      0     0    0            0   
30  31   18        24       0     0        0      0     0    0            0   

    ...  travel  writing  art  game  history  language  movies  music  \
4   ...       0        0    1     0        0         1       1      0   
30  ...       0        0    0     0        0         1       1      0   

    politics  cluster  
4          0       11  
30         0       11  

[2 rows x 25 columns]


In [17]:
print(data[data['cluster'] == 8])

    id  age  angkatan  gender  code  cooking  dance  golf  gym  gym cooking  \
8    9   18        24       0     0        1      1     0    0            0   
12  13   18        23       0     0        1      0     0    0            0   
22  23   17        24       0     0        1      1     0    0            0   
28  29   18        24       0     0        0      1     0    0            0   
32  33   18        24       0     0        0      1     0    0            0   
43  44   18        24       0     0        0      1     0    0            0   

    ...  travel  writing  art  game  history  language  movies  music  \
8   ...       0        1    0     0        0         0       1      0   
12  ...       0        1    0     0        0         0       1      0   
22  ...       1        0    0     1        0         0       1      1   
28  ...       1        0    0     0        1         0       1      0   
32  ...       0        1    0     1        0         0       0      0   
43  ... 

In [18]:
print(data[data['cluster'] == 6])

    id  age  angkatan  gender  code  cooking  dance  golf  gym  gym cooking  \
3    4   18        24       0     1        0      0     0    0            0   
24  25   18        24       0     0        0      0     0    0            0   
40  41   18        24       0     1        0      0     0    0            0   
48  49   18        24       1     1        0      0     0    0            0   

    ...  travel  writing  art  game  history  language  movies  music  \
3   ...       0        1    0     0        1         0       0      0   
24  ...       0        0    1     0        0         0       0      0   
40  ...       0        0    1     0        0         0       0      0   
48  ...       0        0    1     1        0         0       0      0   

    politics  cluster  
3          1        6  
24         0        6  
40         0        6  
48         0        6  

[4 rows x 25 columns]
